##关于协程的知识##

async定义协程对象，一个没有马上执行的程序体
asyncio.get_event_loop() 创建事件循环
再把协程对象丢给事件循环管理运行loop.run_until_complete(async_object)
await  挂起协程等待另一个完成
task   创建一个任务，同意管理 loop.create_task(async_object)

等待结束 loog.run_util_complete(task)

future和task没有本质区别
future_task = asyncio.ensure_future(c) 




In [4]:
import asyncio

def main():
    async def test1():
        print('1')
        await test2()#等待test2完成后运行
        print('2')

    async def test2():
        print('3')
        print('4')

 

    loop = asyncio.get_event_loop()
    loop.run_until_complete(test1())

#if __name__ == '__main__':
#   main()

##多任务协作

这里的tasks,表示一个任务序列(通常为列表)
可以使用 run_until_complete(asyncio.wait(tasks))
注册多个任务也可以使用run_until_complete(asyncio. gather(*tasks))

在PyCharm中能够正确运行，在jupyter notebook中确报错，因为jupyter已经运行了一个event loop。
    RuntimeError: This event loop is already running。


解决办法具体如下：
1.pip3 install nest_asyncio
2.在协程代码开头，加上如下两行：

#允许协程嵌套
import nest_asyncio
nest_asyncio.apply()

In [14]:
import asyncio, time
import nest_asyncio

nest_asyncio.apply()

async def do_some_work(i, n): #使用async关键字定义异步函数
  print('任务{}等待: {}秒'.format(i, n))
  await asyncio.sleep(n) #休眠一段时间
  return '任务{}在{}秒后返回结束运行'.format(i, n)

start_time = time.time() #开始时间

loop = asyncio.get_event_loop()
tasks = [asyncio.create_task(do_some_work(1, 2)),
        asyncio.create_task(do_some_work(2, 1)),
        asyncio.create_task(do_some_work(3, 3))]


loop.run_until_complete(asyncio.gather(*tasks))

for task in tasks:
  print('任务执行结果: ', task.result()) 

print('运行时间: ', time.time() - start_time)

任务1等待: 2秒
任务2等待: 1秒
任务3等待: 3秒
任务执行结果:  任务1在2秒后返回结束运行
任务执行结果:  任务2在1秒后返回结束运行
任务执行结果:  任务3在3秒后返回结束运行
运行时间:  3.0096611976623535


![https://pic.rmb.bdstatic.com/bjh/down/71b2022fc45aa8195a1fc73ee3b8f957.png]
试试，爬虫爬英雄的皮肤

1、通过查看network发现对应的皮肤数据在js文件里;
2、然后我们发现了英雄皮肤存放的url链接规律:'https://game.gtimg.cn/images/lol/act/img/js/hero/英雄id.js'
3、这里我们就爬 取前20个英雄的皮肤; id = 1-21 
4、现在我们可以爬取全部英雄。

In [22]:
import requests
import json
#通过这个链接获取全部英雄的数据，我们取出其中的英雄ID，用于下一个函数。
headers = ""  
res = requests.get("http://game.gtimg.cn/images/lol/act/img/js/heroList/hero_list.js",headers=headers)
result = res.content.decode('utf-8')
res_dict = json.loads(result)

heroId = []
for h in res_dict['hero']:
    heroId.append(h['heroId'])
#heroId = res_dict["heroId"]

print(heroId)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '48', '50', '51', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '67', '68', '69', '72', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '89', '90', '91', '92', '96', '98', '99', '101', '102', '103', '104', '105', '106', '107', '110', '111', '112', '113', '114', '115', '117', '119', '120', '121', '122', '126', '127', '131', '133', '134', '136', '141', '142', '143', '145', '147', '150', '154', '157', '161', '163', '164', '166', '200', '201', '202', '203', '221', '222', '223', '234', '235', '236', '238', '240', '245', '246', '254', '266', '267', '268', '350', '360', '412', '420', '421', '427', '429', '432', '497', '498', '516', '517', '518', '523', '526', '555', '711', '777', 

In [ ]:
import asyncio, time,os
import nest_asyncio
import requests
import json

nest_asyncio.apply()

#. 获取英雄皮肤ulr地址：
#前面的英雄id是按顺序的所有可以使用range(1,21),动态构造url;
def get_page():
  headers = ""  
  res = requests.get("http://game.gtimg.cn/images/lol/act/img/js/heroList/hero_list.js",headers=headers)
  result = res.content.decode('utf-8')
  res_dict = json.loads(result)

  heroId = []
  for h in res_dict['hero']:
    heroId.append(h['heroId'])
  #heroId = res_dict["heroId"]

  #print(heroId)
  



  page_urls = [] 
  for i in heroId:#range(1,21):
      url = 'https://game.gtimg.cn/images/lol/act/img/js/hero/{}.js'.format(i)
      print(url)
      page_urls.append(url) 
  return page_urls


#2. 请求每一页的url地址
#并对网页进行解析获取皮肤图片的url地址:
headers = ""

def get_img():
  img_urls = [] 
  page_urls = get_page() 
  for page_url in page_urls:
      res = requests.get(page_url, headers=headers)
      result = res.content.decode('utf-8')
      res_dict = json.loads(result)
      skins = res_dict["skins"]

      for hero in skins:
        item = {}
        item['name'] = hero["heroName"]
        item['skin_name'] = hero["name"]
        if hero["mainImg"] == '':
          continue
        item['imgLink'] = hero["mainImg"]
        print(item)
        img_urls.append(item)
  return img_urls

#3. 创建协程函数
#这里我们根据英雄名创建文件夹,然后就是注意图片的命名,不要忘记/,目录结构确立
async def save_img(index, img_url):
    path = "皮肤/" + img_url['name']
    if not os.path.exists(path):
        os.makedirs(path)
    content = requests.get(img_url['imgLink'], headers=headers).content
    with open('./皮肤/' + img_url['name'] + '/' + img_url['skin_name'] + str(index) + '.jpg', 'wb') as f:
        f.write(content)

def main():
    loop = asyncio.get_event_loop() 
    img_urls = get_img() 
    print(len(img_urls)) 
    tasks = [save_img(img[0], img[1]) for img in enumerate(img_urls)] 
    try:
        loop.run_until_complete(asyncio.wait(tasks)) 
    finally:
        loop.close()#jupyte会报错，正常环境不会。

if __name__ == '__main__':
    start = time.time() 
    main() 
    end = time.time() 
    print(end - start)        